In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime, math

In [2]:
%%capture
mta = pd.read_csv('../../data/mta_1708.csv', error_bad_lines=False)
# mta.set_index('PublishedLineName', inplace=True)
m100 = mta.loc[(mta['PublishedLineName']== 'M100') & (mta['DestinationName'] == 'INWOOD 220 ST via AMSTERDAM via BWAY'),]

In [5]:
arrivals = m100.loc[m100['ArrivalProximityText']=='at stop']
arrivals.head(20)

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
1523,2017-08-01 00:11:39,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_4349,40.859447,-73.922601,10 AV/HARLEM RIVER DR,at stop,18.0,2017-08-01 00:11:52,24:08:48
1762,2017-08-01 00:21:06,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_4368,40.840707,-73.939571,ST NICHOLAS AV/W 167 ST,at stop,25.0,2017-08-01 00:21:16,24:14:01
3232,2017-08-01 00:41:29,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8375,40.804519,-73.937764,E 125 ST/LEXINGTON AV,at stop,3.0,NaN,24:38:03
3850,2017-08-01 00:51:52,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8375,40.814355,-73.955555,AMSTERDAM AV/W 126 ST,at stop,2.0,NaN,24:47:36
4423,2017-08-01 01:01:11,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8375,40.834044,-73.941169,AMSTERDAM AV/W 159 ST,at stop,21.0,NaN,24:58:27
13699,2017-08-01 05:21:36,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8368,40.807974,-73.945989,W 125 ST/MALCOLM X BL,at stop,0.0,2017-08-01 05:21:58,05:18:18
13700,2017-08-01 05:21:36,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8368,40.807974,-73.945989,W 125 ST/MALCOLM X BL,at stop,0.0,2017-08-01 05:21:58,05:18:52
13701,2017-08-01 05:21:36,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8368,40.807974,-73.945989,W 125 ST/MALCOLM X BL,at stop,0.0,2017-08-01 05:21:58,05:19:27
14605,2017-08-01 05:31:39,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8368,40.825603,-73.947335,AMSTERDAM AV/W 145 ST,at stop,3.0,2017-08-01 05:31:56,05:29:02
14606,2017-08-01 05:31:39,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8368,40.825603,-73.947335,AMSTERDAM AV/W 145 ST,at stop,3.0,2017-08-01 05:31:56,05:30:42


In [3]:
new = arrivals.loc[:,['RecordedAtTime','VehicleRef']].sort_values(['VehicleRef', 'RecordedAtTime'], ascending=True)
uniqueRef = new['VehicleRef'].unique()
new.RecordedAtTime = pd.to_datetime(new.RecordedAtTime)

## Setting the time deltas only within the same vehicle refs
https://stackoverflow.com/questions/20648346/computing-diffs-within-groups-of-a-dataframe


In [4]:
new.sort_values(['VehicleRef', 'RecordedAtTime'], inplace=True)
new['diffs'] = new['RecordedAtTime'].diff()
mask = new.VehicleRef != new.VehicleRef.shift(1)
new['diffs'][mask] = np.nan
new

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,RecordedAtTime,VehicleRef,diffs
5010109,2017-08-23 10:01:19,NYCT_1202,NaT
5012297,2017-08-23 10:11:48,NYCT_1202,0 days 00:10:29
5044476,2017-08-23 12:51:31,NYCT_1202,0 days 02:39:43
5046521,2017-08-23 13:02:02,NYCT_1202,0 days 00:10:31
5048584,2017-08-23 13:11:27,NYCT_1202,0 days 00:09:25
5057010,2017-08-23 13:51:35,NYCT_1202,0 days 00:40:08
5090596,2017-08-23 16:12:27,NYCT_1202,0 days 02:20:52
5101690,2017-08-23 16:52:25,NYCT_1202,0 days 00:39:58
1394003,2017-08-07 09:33:42,NYCT_1206,NaT
1402955,2017-08-07 10:13:36,NYCT_1206,0 days 00:39:54


In [ ]:
new.dropna()
new = new[new.diffs > '02:00:00']
new

mta.describe()